In [20]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

import warnings
warnings.filterwarnings('ignore')

from openpyxl import load_workbook
from datetime import date
from datetime import datetime
from dateutil.relativedelta import relativedelta
from dateutil.rrule import rrule, DAILY
from tqdm import tqdm

In [21]:
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

### Functions

#### get data

In [22]:
import pymssql  
conn = pymssql.connect(server='10.220.8.32', 
                       user='ptuser',
                       password='@Yuanta', 
                       database='PacketTimeStamp')
cursor = conn.cursor()

In [23]:
def daily_data_r(date, groupid):
    date = date
    query = f"""SELECT 
             [Pid]
            ,[GroupId]
            ,[BsSign]
            ,[TxTime]
            ,[TimePart]
            ,[TimeStamp]
            ,[BidQty1]
            ,[Bid1]
            ,[Ask1]
            ,[AskQty1]
            ,[DealPrice]
            ,[DealQty]
            ,[Type]
            ,[SeqNo]
            ,[MarketCode]
            FROM [TickData].[dbo].[tblTickData_TAIFEX_Roll_{date}]
            where groupid ='{groupid}'
            """
    cursor.execute(query)
    columns = ['Pid', 'GroupId', 'BsSign', 'TxTime', 'TimePart', 'TimeStamp', 'BidQty1', 'Bid1', 'Ask1', 'AskQty1', 'DealPrice', 'DealQty', 'Type', 'SeqNo', 'MarketCode']
    roll = pd.DataFrame(columns=columns)
    for row in cursor:
        roll.loc[len(roll)] = list(row)
    try:
        roll = roll.drop(0)
    except:
        return 
    roll['TxTime'] = pd.to_datetime(roll['TxTime'])
    roll = roll.sort_values(['Pid', 'TxTime'])
    roll = roll.reset_index(drop=True)
    return roll

In [24]:
def get_key(my_dict, a):
    key_list = list(my_dict.keys())
    val_list = list(my_dict.values()) 
    # print key with val 100
    position = val_list.index(a)
    return key_list[position]

In [25]:
def find_wed(start): # input 20230811
    first_day = start.replace(day=1) # 20230801
    for i in range(7):
        if (first_day + timedelta(days=i)).isoweekday() == 3:
            x = first_day + timedelta(days=i) 
            # x = 20230802 是第一個星期三
        else:
            continue
    for i in range(6):
        if start <= x + timedelta(days=7)*i:
            return i+1
        else:
            continue

In [26]:
def daily_data_f(date):
    TickData = pd.DataFrame()
    date2 = datetime.strptime(date, '%Y%m%d')
    month_code = {'A':1, 'B':2, 'C':3, 'D':4, 'E':5, 'F':6, 'G':7, 'H':8, 'I':9, 'J':10, 'K':11, 'L':12}
    code = get_key(month_code, int(date[4:6]))
    if find_wed(date2)>3:
        code = get_key(month_code, int(date[4:6])+1)
    query = f"SELECT [TxTime], [Pid], [Bid1], [Ask1] FROM [TickData].[dbo].[tblTickData_TAIFEX_{date}] where Pid = 'TXF{code}3'"
    TickData = pd.concat([TickData, pd.read_sql(query, conn)], axis = 0)
    TickData['TxTime'] = pd.to_datetime(TickData['TxTime'])
    TickData = TickData.drop_duplicates(keep='first')
    TickData = TickData.sort_values(['TxTime', 'Bid1', 'Ask1']).reset_index(drop=True)
    # delete all rows with column 'Age' has value 30 to 40
    indexba0 = TickData[ (TickData['Bid1'] == '0') | (TickData['Ask1']== '0') ].index
    TickData.drop(indexba0 , inplace=True)
    return TickData

#### calculation

##### 沒相關係數

In [27]:
def calculation_2(roll):
    roll['買賣寬'] = (roll['Bid1'] - roll['Ask1']).abs()
    roll['流動差'] = np.where(roll['買賣寬'] > 30, 1, np.nan)
    roll['流動好'] = np.where(roll['買賣寬'] > 30, np.nan, 1)
    roll['買賣寬*流動好'] = roll['買賣寬']*roll['流動好']
    def in_amount(roll):
        if roll['DealPrice'] <= roll['Bid1']:
            return roll['DealQty']
        else:
            return
    roll['內盤量'] = roll.apply(in_amount, axis = 1)
    roll['內盤金額'] = roll['內盤量']*roll['DealPrice']
    roll['外盤量'] = np.where(roll['DealPrice'] >= roll['Ask1'], roll['DealQty'], np.nan)
    roll['外盤金額'] = roll['外盤量']*roll['DealPrice']
    def excess_in_amount(roll):
        if roll['Type'] == 'Roll':
            return 0
        if roll['DealPrice'] < roll['Bid1']:
            return roll['DealQty']
        else:
            return 0
    roll['超額內量'] = roll.apply(excess_in_amount, axis = 1)
    def excess_out_amount(roll):
        if roll['Type'] == 'Roll':
            return 0
        if roll['DealPrice'] > roll['Ask1']:
            return roll['DealQty']
        else:
            return 0
    roll['超額外量'] = roll.apply(excess_out_amount, axis = 1)
    def bid_liq_good(roll):
        return roll['Bid1']*roll['流動好']
    roll['Bid*流動好'] = roll.apply(bid_liq_good, axis = 1)
    def Ask_liq_good(roll):
        return roll['Ask1']*roll['流動好']
    roll['Ask*流動好'] = roll.apply(Ask_liq_good, axis = 1)
    def excess_gap(row):
        if row['DealPrice'] < row['Bid1']:
            return (row['Bid1'] - row['DealPrice'])*row['超額內量']
        elif row['DealPrice'] > row['Ask1']:
            return (row['DealPrice'] - row['Ask1'])*row['超額外量']
        return 0
    roll['超額價差'] = roll.apply(excess_gap, axis=1)
    
    roll['Bid*流動好'] = roll['Bid*流動好'].astype(float)
    roll['Ask*流動好'] = roll['Ask*流動好'].astype(float)

In [28]:
def daily_result_2(roll, date):
    daily_result = []
    def Avg_range(x):
        try:
            return x['買賣寬*流動好'].sum()/x['流動好'].sum()
        except:
            return
    daily_result.append(roll.groupby(['Pid', 'MarketCode']).apply(Avg_range))
    daily_result.append(roll.groupby(['Pid', 'MarketCode'])['流動差'].sum())
    daily_result.append(roll.groupby(['Pid', 'MarketCode'])['內盤量'].sum())
    def divid_df_in(x):
        try:
            return x['內盤金額'].sum()/x['內盤量'].sum()
        except:
            return
    daily_result.append(roll.groupby(['Pid', 'MarketCode']).apply(divid_df_in))
    daily_result.append(roll.groupby(['Pid', 'MarketCode'])['外盤量'].sum())
    def divid_df_out(x):
        try:
            return x['外盤金額'].sum()/x['外盤量'].sum()
        except:
            return
    daily_result.append(roll.groupby(['Pid', 'MarketCode']).apply(divid_df_out))   
    daily_result.append(roll.groupby(['Pid', 'MarketCode']).apply(lambda x: x['Ask*流動好'].max() - x['Bid*流動好'].min()))
    daily_result.append(roll.groupby(['Pid', 'MarketCode']).apply(lambda x: x['超額內量'].sum() + x['超額外量'].sum()))
    daily_result.append(roll.groupby(['Pid', 'MarketCode'])['超額價差'].sum())
    
    daily_result_df = pd.concat(daily_result, axis = 1).reset_index()
    daily_result_df.insert(0, '日期', date)
    columns2 = ['日期', 'Pid', '日夜盤','買賣寬平均', '流動不好筆數', '內盤量', '內盤均價', '外盤量', '外盤均價', 'roll震幅', '超額總口數', '超額總點數']
    daily_result_df.columns = columns2
    return daily_result_df

In [29]:
def daily_result_2(roll, date):
    daily_result = []
    daily_result.append(roll.groupby(['Pid', 'MarketCode'])['買賣寬*流動好'].sum())
    daily_result.append(roll.groupby(['Pid', 'MarketCode'])['流動好'].sum())
 
    daily_result_df = pd.concat(daily_result, axis = 1).reset_index()
    daily_result_df.insert(0, '日期', date)
    columns2 = ['日期', 'Pid', '日夜盤','買賣寬總和', '流動好筆數']
    daily_result_df.columns = columns2
    return daily_result_df

##### 有相關係數

In [30]:
def calculation(roll):
    roll['買賣寬'] = np.where(roll['BidQty1'] == 0, 31, (roll['Bid1_x'] - roll['Ask1_x']).abs())
    roll['流動差'] = np.where(roll['買賣寬'] > 30, 1, 0)
    roll['流動好'] = np.where(roll['買賣寬'] > 30, 0, 1)
    def in_amount(roll):
        if roll['DealPrice'] <= roll['Bid1_x']:
            return roll['DealQty']
        else:
            return 0
    roll['內盤量'] = roll.apply(in_amount, axis = 1)
    def out_amount(roll):
        if roll['DealPrice'] >= roll['Ask1_x']:
            return roll['DealQty']
        else:
            return 0
    roll['外盤量'] = roll.apply(out_amount, axis = 1)
    roll['內外盤量差'] = roll['外盤量'] - roll['內盤量']
    def Avg_BA(roll):
        if roll['AskQty1'] == 0:
            return 
        if roll['BidQty1'] == 0:
            return 
        else:
            return (roll['Bid1_x'] + roll['Ask1_x'])/2
    roll['Avg_BA'] = roll.apply(Avg_BA, axis = 1)
    roll['diff'] = roll.groupby(['Pid_x', 'MarketCode'])[['Avg_BA']].diff()
    def sign(row):
        if pd.isnull(row['diff']) or row['diff'] == 0:
            return 0
        return row['流動好']
    roll['sign'] = roll.apply(sign, axis = 1)
    roll['Avg_BA_f'] = np.where(roll['Bid1_y']==0, np.nan, (roll['Bid1_y'] + roll['Ask1_y'])/2)
    
    roll['Avg_BA'] = roll['Avg_BA'].astype(float)
    roll['Avg_BA_f'] = roll['Avg_BA_f'].astype(float)

    def cul_dev_f(roll):
        cul_dev = []
        for i in range(len(roll)):
            if i == 0:
                mc = roll.loc[i, 'MarketCode']
                cum = roll.loc[i, '內外盤量差']
                continue
            if mc != roll.loc[i, 'MarketCode']:
                mc = roll.loc[i, 'MarketCode']
                cum = roll.loc[i, '內外盤量差']
                continue
            cum += roll.loc[i, '內外盤量差']
            if roll.loc[i, 'diff'] != 0:
                cul_dev.append(cum)
                cum = 0
        return cul_dev
    cul_dev = cul_dev_f(roll)

    roll2 = roll[roll['diff']!=0].reset_index(drop = True)
    roll2['diff_f'] = roll2['Avg_BA_f'].diff()
    roll2 = roll2.dropna()
    roll2.insert(29, '累積量差', cul_dev)
    return roll2

In [31]:
def daily_result(roll, input_month, groupid1):
    daily_result = []
    def corr_f(x):
        m = x['diff'][x['sign']==1]
        n = x['diff_f']
        return m.corr(n)
    daily_result.append(roll.groupby(['Pid_x', 'MarketCode']).apply(corr_f))
    def corr_a(x):
        m = x['diff'][x['sign']==1]
        n = x['累積量差']
        return m.corr(n)
    daily_result.append(roll.groupby(['Pid_x', 'MarketCode']).apply(corr_a))
    
    daily_result_df = pd.concat(daily_result, axis = 1).reset_index()
    daily_result_df.insert(0, '月', input_month)
    daily_result_df.insert(0, 'GroupID', groupid1)
    columns2 = ['GroupID', '月', 'Pid', '日夜盤','期貨相關性', '盤量相關性']
    daily_result_df.columns = columns2
    return daily_result_df

#### 處理代碼

In [32]:
import calendar
def find_last_day(now):
    now = datetime.strptime(now, '%Y%m%d')
    last = calendar.monthrange(now.year, now.month)[1]
    last_day = now.replace(day=last)
    return last_day

now = '20230708'
find_last_day(now)

datetime.datetime(2023, 7, 31, 0, 0)

In [33]:
from datetime import datetime, timedelta

def find_wed(start):
    first_day = start.replace(day=1)
    for i in range(7):
        if (first_day + timedelta(days=i)).isoweekday() == 3:
            x = first_day + timedelta(days=i)
        else:
            continue
    for i in range(6):
        if start <= x + timedelta(days=7)*i:
            return i+1
        else:
            continue

now = '20230831'
now = datetime.strptime(now, '%Y%m%d')
find_wed(now)

6

In [34]:
month_code = {'A':'01', 'B':'02', 'C':'03', 'D':'04', 'E':'05', 'F':'06', 'G':'07', 'H':'08', 'I':'09', 'J':'10', 'K':'11', 'L':'12'}

In [35]:
def Pid_class(a, time):
    time2 = datetime.strptime(time, '%Y%m%d')
    if a[2] == 'F':#月合約
        now = time2.month
        input = int(month_code[a[3]])
        if find_wed(time2) > 3:
            now = now + 1
        if int(a[4]) > int(time[3]):
            input = input +12
        val = input - now
        if val < 3:
            return 'C'+ f'{val + 1}'
        if input%3 == 0: # 6, 9, 12
            if val < 6:
                return 'C4'
            elif val < 9:
                return 'C5'
            elif val < 12:
                return 'C6'
            else:
                return 'others'
        else:
            return 'others'

    else: #週合約
        if a[4] == time[3]: #同年
            if month_code[a[3]] == time[4:6]: #同月
                return 'W' + f'{int(a[2]) - find_wed(time2) + 1}'
            elif int(month_code[a[3]]) == time2.month + 1:
                if find_wed(find_last_day(time)) == 5:
                    return 'W' + f'{int(a[2]) - find_wed(time2) + 5}'
                if find_wed(find_last_day(time)) == 6:
                    return 'W' + f'{int(a[2]) - find_wed(time2) + 6}'
        else: # 12月跨1月
            if int(month_code[a[3]]) + 12 == time2.month + 1:
                return 'W' + f'{int(a[2]) - find_wed(time2) + 5}'
a = 'MXFK3'#'MX1F3'
time = '20230719'#'20230530'
Pid_class(a, time)

'others'

In [36]:
def Pid_tran(row):
    l1 = row['Pid'].split('/')
    time = row['日期']
    left = l1[0]
    right = l1[1]
    if len(row['Pid']) == 8: # 月/月
        right = 'MXF' + right
    return Pid_class(left, time) + '/' + Pid_class(right, time)

#### merge data

In [37]:
def delete_bad_rows(roll):    
    roll['Bid1'] = roll['Bid1'].astype(float)
    roll['Ask1'] = roll['Ask1'].astype(float)
    roll['DealPrice'] = roll['DealPrice'].astype(float)
    roll = roll[roll['BidQty1']!= 0]
    roll = roll[roll['AskQty1']!= 0]
    roll = roll[roll['Bid1'].map(float.is_integer)]
    roll = roll[roll['Ask1'].map(float.is_integer)]
    return roll

In [38]:
def only_c1c2(roll):
    def only_date(row):
        val = str(row['TxTime'])[:10]
        l1 = val.split('-')
        return l1[0]+l1[1]+l1[2]
    roll['日期'] = roll.apply(only_date, axis = 1)
    roll.insert(1, "Pid_trans", roll.apply(Pid_tran, axis=1))
    return roll[roll['Pid_trans']=='C1/C2'], roll[roll['Pid_trans']!='C1/C2']

In [39]:
def daily_data_all(roll, TickData): 
    roll2 = pd.merge_asof(roll.sort_values('TxTime'), TickData.sort_values('TxTime'), on = 'TxTime')
    roll2.sort_values(['Pid_x', 'TxTime'])
    roll2 = roll2.reset_index(drop=True)
    columns = ['Pid_x', 'Pid_trans', 'Pid_y', 'GroupId', 'BsSign', 'TxTime', 'TimePart',
               'TimeStamp', 'BidQty1', 'Bid1_x', 'Ask1_x', 'AskQty1', 'DealPrice',
               'DealQty', 'Type', 'MarketCode', 'Bid1_y', 'Ask1_y']
    roll2 = roll2[columns]
    roll2['Bid1_y'] = pd.to_numeric(roll2['Bid1_y'])
    roll2['Ask1_y'] = pd.to_numeric(roll2['Ask1_y'])
    return roll2

### Main

##### 沒相關係數

In [ ]:
from pandas import ExcelWriter
w = ExcelWriter('roll_result0816.xlsx')
date_range = pd.date_range(start = '20230201', end = '20230706')
for groupid1 in ['MX', 'TX']:
    roll_result = []
    roll_result_df = pd.DataFrame()
    for i in date_range:
        try:
            date2 = i.strftime('%Y%m%d')
            daily_roll = daily_data_r(date2, groupid1)
            if daily_roll.empty:
                continue
            daily_roll = delete_bad_rows(daily_roll)
            calculation_2(daily_roll)
            daily_result_df = daily_result_2(daily_roll, date2)
            roll_result.append(daily_result_df)
        except:
            continue
    roll_result_df = pd.concat(roll_result, axis = 0)
    roll_result_df.insert(2, "Pid_trans", roll_result_df.apply(Pid_tran, axis=1))
    roll_result_df = roll_result_df.sort_values(['日期', 'Pid_trans']).reset_index(drop=True)
    roll_result_df.to_excel(w, sheet_name=f'{groupid1}')
w.save()
w.close()
# """ 

In [ ]:
#""" test
date1 = '20230615'
groupid1 = 'MX'
daily_roll = daily_data_r(date1, groupid1)
daily_roll = delete_bad_rows(daily_roll)
daily_roll_c1c2, daily_roll_other = only_c1c2(daily_roll)
calculation_2(daily_roll_c1c2)
daily_result_df = daily_result_2(daily_roll_c1c2, date1)
daily_result_df
#wide = daily_result_df.groupby(['Pid', 'MarketCode'])['買賣寬總和'].sum()
#liquid = daily_result_df['流動好筆數'].sum()
#daily_roll[daily_roll['Pid']=='MXFC3/L3']
#daily_roll[daily_roll['Bid1'].map(float.is_integer)]
#daily_result_df[daily_result_df['Pid']=='MXFC3/L3']
# """

In [ ]:
day_wide_c1c2 = daily_result_df.loc[0, '買賣寬總和']
day_liquid_c1c2 = daily_result_df.loc[0, '流動好筆數']
night_wide_c1c2 = daily_result_df.loc[1, '買賣寬總和']
night_liquid_c1c2 = daily_result_df.loc[1, '流動好筆數']
[day_wide_c1c2, day_liquid_c1c2, night_wide_c1c2, night_liquid_c1c2]

In [ ]:
calculation_2(daily_roll_other)
daily_result_df2 = daily_result_2(daily_roll_other, date1)
daily_result_df2 

In [ ]:
day_wide_other = daily_result_df2.groupby(['日夜盤'])['買賣寬總和'].sum()[0]
night_wide_other = daily_result_df2.groupby(['日夜盤'])['買賣寬總和'].sum()[1]
day_liquid_other = daily_result_df2.groupby(['日夜盤'])['流動好筆數'].sum()[0]
night_liquid_other = daily_result_df2.groupby(['日夜盤'])['流動好筆數'].sum()[1]

In [ ]:
daily_result_df2.groupby(['日夜盤'])['流動好筆數'].sum()

##### 有相關係數

In [ ]:
from pandas import ExcelWriter
from openpyxl import load_workbook
from tqdm import tqdm

date_range = pd.date_range(start = '20230601', end = '20230721')
# 0601~0706, 
for groupid1 in tqdm(['MX', 'TX']):
    roll_result = []
    roll_result_df = pd.DataFrame()
    for i in date_range:
        try:
            date2 = i.strftime('%Y%m%d')
            daily_roll = daily_data_r(date2, groupid1)
            if daily_roll.empty:
                continue
            daily_roll = delete_bad_rows(daily_roll)
            TickData = daily_data_f(date2)
            daily_roll = only_c1c2(daily_roll)
            daily_roll2 = daily_data_all(daily_roll, TickData)
            daily_roll3 = calculation(daily_roll2)
            daily_result_df = daily_result(daily_roll3, date2)
            roll_result.append(daily_result_df)
        except:
            continue
    roll_result_df = pd.concat(roll_result, axis = 0)
    roll_result_df.insert(2, "Pid_trans", roll_result_df.apply(Pid_tran, axis=1))
    roll_result_df = roll_result_df.sort_values(['日期', 'Pid_trans']).reset_index(drop=True)
    # roll_result_df.to_excel(w, sheet_name=f'{groupid1}')

    filename = 'roll_result0816cor.xlsx'
    workbook = load_workbook(filename)
    writer = pd.ExcelWriter(filename, engine='openpyxl')
    writer.book = workbook
    writer.sheets = {ws.title: ws for ws in workbook.worksheets}
    roll_result_df.to_excel(writer, startrow=writer.sheets[f'{groupid1}'].max_row, index = False, header= False, sheet_name=f'{groupid1}')
    writer.close()
# """ 

In [ ]:
#""" 範例
date2 = '20230328'
groupid1 = 'TX'
roll_result = []
daily_roll = daily_data_r(date2, groupid1)
daily_roll = delete_bad_rows(daily_roll)
daily_roll = only_c1c2(daily_roll)
TickData = daily_data_f(date2)
daily_roll2 = daily_data_all(daily_roll, TickData)
calculation(daily_roll2)

daily_result_df, daily_roll3 = daily_result(daily_roll2, date2)
daily_roll2[daily_roll2['MarketCode']==1][daily_roll2['diff']!=0]
# daily_result_df
#"""

In [ ]:
daily_roll2[daily_roll2['MarketCode']==1]

In [ ]:
from pandas import ExcelWriter
w2 = ExcelWriter(f'raw{date2}_{groupid1}0808.xlsx')
daily_roll2.to_excel(w2, 'raw data')
daily_roll3.to_excel(w2, 'diff!=0')
daily_result_df.to_excel(w2, 'correlation result')
w2.save()
w2.close()

##### Monthly and last week correlation

In [43]:
def third_wed(month:str):
    for i in range(7,25):
        date = month
        date = date + str(i)
        date2 = datetime.strptime(date, '%Y%m%d')
        if find_wed(date2) == 3:
            return date2
def find_period(month:str):
    day1 = datetime.strptime(month, '%Y%m')
    date_after_month = day1 - relativedelta(months=1)
    last_month = str(date_after_month)[:7]. replace('-', '')
    begin = third_wed(last_month)+timedelta(days=7)
    end = third_wed(month)+timedelta(days=6)
    return begin, end
def find_lastweek(month:str):
    day1 = datetime.strptime(month, '%Y%m')
    begin = third_wed(month)
    end = third_wed(month)+timedelta(days=6)
    return begin, end

In [ ]:
#input_month = '202305'
method = find_lastweek # find_period
for input_month in tqdm(['202301', '202302', '202303', '202304', '202305', '202306', '202307']):
    beg = method(input_month)[0].strftime("%Y%m%d")
    end = method(input_month)[1].strftime("%Y%m%d")

    date_range = pd.date_range(start = beg, end = end)
    roll_result = pd.DataFrame()
    for groupid1 in ['MX', 'TX']:
        daily_roll_l = []
        roll_result_df = pd.DataFrame()
        for i in date_range:
            try:
                date2 = i.strftime('%Y%m%d')
                daily_roll = daily_data_r(date2, groupid1)
                if daily_roll.empty:
                    continue
                daily_roll = delete_bad_rows(daily_roll)
                TickData = daily_data_f(date2)
                daily_roll = only_c1c2(daily_roll)
                daily_roll2 = daily_data_all(daily_roll, TickData)
                daily_roll3 = calculation(daily_roll2)
                daily_roll_l.append(daily_roll3)
            except:
                continue
        daily_roll_df = pd.concat(daily_roll_l, axis = 0).sort_values(['GroupId', 'TxTime', 'MarketCode'])
        monthly_result_df = daily_result(daily_roll_df, input_month, groupid1)
        roll_result = pd.concat([roll_result,  monthly_result_df], axis = 0)

    filename = "roll_correlation.xlsx"
    workbook = load_workbook(filename)
    writer = pd.ExcelWriter(filename, engine='openpyxl')
    writer.book = workbook
    writer.sheets = {ws.title: ws for ws in workbook.worksheets}
    roll_result.to_excel(writer, startrow=writer.sheets['weekly'].max_row, index = False, header= False, sheet_name='weekly')
    writer.close()

In [42]:
def divide(x, y):
    try:
        return x/y
    except:
        return 0

In [44]:
#input_month = '202305'
method = find_period
result = []

for input_month in tqdm(['202307', '202308']):
    beg = method(input_month)[0].strftime("%Y%m%d")
    end = method(input_month)[1].strftime("%Y%m%d")

    date_range = pd.date_range(start = beg, end = end)
    roll_result = pd.DataFrame()
    for groupid1 in ['MX', 'TX']:
        roll_result_df = pd.DataFrame()
        day_wide_c1c2 = 0
        day_wide_other = 0
        night_wide_c1c2 = 0
        night_wide_other = 0

        day_liquid_c1c2 = 0
        day_liquid_other = 0
        night_liquid_c1c2 = 0
        night_liquid_other = 0

        for i in date_range:
            date2 = i.strftime('%Y%m%d')
            try:
                daily_roll = daily_data_r(date2, groupid1)
            except:
                continue
            if daily_roll is None:
                continue
            daily_roll = delete_bad_rows(daily_roll)
            daily_roll_c1c2, daily_roll_other = only_c1c2(daily_roll)

            # c1/c2
            if daily_roll_c1c2.empty:
                pass
            else:
                calculation_2(daily_roll_c1c2)
                daily_result_df = daily_result_2(daily_roll_c1c2, date2)
            try:
                day_wide_c1c2 += daily_result_df.loc[0, '買賣寬總和']
            except:
                pass
            try:
                day_liquid_c1c2 += daily_result_df.loc[0, '流動好筆數']
            except:
                pass
            try:
                night_wide_c1c2 += daily_result_df.loc[1, '買賣寬總和']
            except:
                pass
            try:
                night_liquid_c1c2 += daily_result_df.loc[1, '流動好筆數']
            except:
                pass

            # non c1/c2
            calculation_2(daily_roll_other)
            daily_result_df2 = daily_result_2(daily_roll_other, date2)
            try:
                day_wide_other += daily_result_df2.groupby(['日夜盤'])['買賣寬總和'].sum()[0]
            except:
                pass
            try:
                day_liquid_other += daily_result_df2.groupby(['日夜盤'])['流動好筆數'].sum()[0]
            except:
                pass
            try:
                night_wide_other += daily_result_df2.groupby(['日夜盤'])['買賣寬總和'].sum()[1]
            except:
                pass
            try:
                night_liquid_other += daily_result_df2.groupby(['日夜盤'])['流動好筆數'].sum()[1]
            except:
                pass

        day_c1c2 = divide(day_wide_c1c2,day_liquid_c1c2)
        night_c1c2 = divide(night_wide_c1c2,night_liquid_c1c2)
        day_other = divide(day_wide_other,day_liquid_other)
        night_other = divide(night_wide_other,night_liquid_other)
        result.append([input_month, groupid1, day_c1c2, day_other, night_c1c2, night_other])
            


100%|██████████| 2/2 [08:20<00:00, 250.06s/it]


In [46]:
pd.DataFrame(result)

,0,1,2,3,4,5
0,202307,MX,2.91,4.67,2.78,4.46
1,202307,TX,1.80,5.45,2.39,5.43
2,202308,MX,2.88,4.63,2.66,4.53
3,202308,TX,1.70,5.38,2.35,5.16
